<a href="https://colab.research.google.com/github/Ahnkyuwon504/AI-modeling/blob/main/BX-gpt-app/embed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 라이브러리 설치

In [1]:
!pip install -q datasets pandas pymongo sentence_transformers
!pip install -q -U transformers
!pip install -q "pymongo[srv]"
# !pip install -q -U bitsandbytes
# !pip install -q PyPDF2 openparse
!pip install -q langchain langchain_experimental langchain_openai
# !pip install -q unstructured > /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu

# 2. Dataset

## 마크다운 문서를 binary 모드로 읽어 디코딩

In [2]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document

# 파일을 바이너리 모드로 읽고 디코딩
def read_file_with_encoding(file_path, encodings=['utf-8', 'latin-1', 'cp1252']):
    for encoding in encodings:
        try:
            with open(file_path, 'rb') as file:
                binary_data = file.read()
                return binary_data.decode(encoding)
        except UnicodeDecodeError as e:
            print(f"Failed to decode with encoding {encoding}: {e}")
    raise ValueError("Unable to decode the file with provided encodings.")

## 개행/svg 파일 제거

In [3]:
import re

def remove_empty_lines(text):
    # 여러 개의 개행 문자를 하나의 개행 문자로 대체
    text = re.sub(r'\n\s*\n', '\n', text)
    return text

def remove_image_lines(text):
    # ![...](...svg) 또는 ![...](...png)로 끝나는 라인
    pattern_svg_jpg = r'^.*!\[.*?\]\(.*?\.(svg|png)\).*$'
    # single hash
    pattern_single_hash = r'^# .*$'
    cleaned_text = re.sub(pattern_svg_jpg, '', text, flags=re.MULTILINE)

    lines = cleaned_text.split('\n')
    cleaned_lines = [line for line in lines if not re.match(pattern_single_hash, line)]
    cleaned_text = '\n'.join(cleaned_lines)
    return cleaned_text

## multi hash/stars 기준 chunking/반복된 hash 제거

In [4]:
def chunk_by_patterns(text):
    # ##, ###, ####, 그리고 **로 시작하고 끝나는 패턴을 기준으로 문서를 청킹
    pattern = r'(\n#{2,4} .*|\n\*\* .* \*\*)'
    # 패턴을 기준으로 분할, 패턴도 포함되도록 분할
    chunks = re.split(pattern, text)
    # 결과 청크를 병합하여 각 청크가 패턴과 내용을 모두 포함하도록 함
    merged_chunks = []
    for i in range(1, len(chunks), 2):
        merged_chunks.append(chunks[i] + chunks[i + 1])
    return merged_chunks

def remove_repeated_triple_hash(chunks):
    cleaned_chunks = []
    for chunk in chunks:
        lines = chunk.split('\n')
        cleaned_lines = []
        triple_hash_count = 0
        for line in lines:
            if line.startswith('###'):
                triple_hash_count += 1
                if triple_hash_count > 1:
                    continue  # 연속된 ### 라인이 5줄 이하인 경우 제거
            else:
                triple_hash_count = 0
            cleaned_lines.append(line)

        cleaned_chunks.append('\n'.join(cleaned_lines))
    return cleaned_chunks

def remove_hashes_and_number_patterns(text):
    # 모든 # 제거 및 숫자.숫자.숫자 형태 제거
    text = text.strip()
    text = re.sub(r'#', '', text)
    text = re.sub(r'\b\d+(\.\d+)+\b', '', text)
    return text

In [5]:
import pandas as pd

df = pd.DataFrame(columns=['text'])

pdf_files = [
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/01-BX-CBP-제품소개.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/02-BX-PF-제품소개.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/12-BX-CBP-공통.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/21-BX-CBP-베이스컴포넌트.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/22-BX-CBP-상품컴포넌트.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/23-BX-CBP-액터컴포넌트.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/24-BX-CBP-계약컴포넌트.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/25-BX-CBP-정산컴포넌트.md',
]

for pdf_file in pdf_files:
    print(f"Start Processing {pdf_file}...")

    curr_markdown_text = read_file_with_encoding(pdf_file)

    curr_data = [Document(page_content=curr_markdown_text)]
    curr_cleaned_text = remove_empty_lines(curr_data[0].page_content)
    curr_cleaned_text = remove_image_lines(curr_cleaned_text)
    curr_cleaned_text = remove_empty_lines(curr_cleaned_text)
    curr_chunks = chunk_by_patterns(curr_cleaned_text)

    final_chunks = remove_repeated_triple_hash(curr_chunks)

    for i, chunk in enumerate(final_chunks):
      cleaned_chunk = remove_hashes_and_number_patterns(chunk)

      if len(cleaned_chunk.strip().split('\n')) == 1:
        continue

      df = pd.concat([df, pd.DataFrame({'text': [cleaned_chunk]})], ignore_index=True)
      # print(cleaned_chunk)
      # print('@@@@@'*10)

print(df.shape)
print(df.head(5))

Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/01-BX-CBP-제품소개.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/02-BX-PF-제품소개.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/12-BX-CBP-공통.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/21-BX-CBP-베이스컴포넌트.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/22-BX-CBP-상품컴포넌트.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/23-BX-CBP-액터컴포넌트.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/24-BX-CBP-계약컴포넌트.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/25-BX-CBP-정산컴포넌트.md...
(434, 1)
                                                text
0   . 금융기관 차세대시스템 구축 연혁\n우리나라 은행 IT 시스템은 1970년대 과...
1   . 차세대시스템 구축 방향성\n차세대시스템 구축을 위해 Framework, Pro...
2   . BX CBP History\n이런 Needs에 맞게 뱅크워어글로벌(BwG)은 ...
3   . BwG 코어뱅킹 아키텍처 Overview\nBwG 코어뱅킹

# 3. Embedding

In [6]:
from sentence_transformers import SentenceTransformer

# embedding_model = SentenceTransformer("thenlper/gte-large")
embedding_model = SentenceTransformer("jhgan/ko-sbert-sts")

def get_embedding(text: str) -> list[float]:
    if not text.strip():
      print("empty")
      return []
    return embedding_model.encode(text).tolist()

df["embedding"] = df["text"].apply(get_embedding)
df.head(5)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

,text,embedding
0,. 금융기관 차세대시스템 구축 연혁\n우리나라 은행 IT 시스템은 1970년대 과...,"[-0.7941996455192566, 0.3398440480232239, -0.1..."
1,". 차세대시스템 구축 방향성\n차세대시스템 구축을 위해 Framework, Pro...","[-1.0208927392959595, -0.0037416480481624603, ..."
2,. BX CBP History\n이런 Needs에 맞게 뱅크워어글로벌(BwG)은 ...,"[-0.13097622990608215, 0.6777176856994629, -0...."
3,. BwG 코어뱅킹 아키텍처 Overview\nBwG 코어뱅킹 아키텍처는 효율적인...,"[-0.17779892683029175, 0.5708097219467163, -0...."
4,. BwG 코어뱅킹 솔루션의 개요\nBwG 코어뱅킹 솔루션은 20년 이상의 코어뱅...,"[-0.21573498845100403, 0.6749694347381592, -0...."


In [10]:
print(len(df['embedding'][0]))

768


# 4. db connection

In [11]:
!curl ipecho.net/plain

34.16.132.144

In [12]:
import pymongo
from google.colab import userdata

def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None

mongo_uri = userdata.get("MONGO_URI")
if not mongo_uri:
    print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

# Ingest data into MongoDB
db = mongo_client["spider504"]
collection = db["240703_movie_collection"]

Connection to MongoDB successful


In [13]:
collection.delete_many({})

documents = df.to_dict("records")
collection.insert_many(documents)

print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


# 5. Vector Search

In [14]:
def vector_search(user_query, collection):
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 150,  # Number of candidate matches to consider
                "limit": 4,  # Return top 4 matches
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "text": 1,
                "score": {"$meta": "vectorSearchScore"},  # Include the search score
            }
        },
    ]

    results = collection.aggregate(pipeline)

    return list(results)

In [15]:
def get_search_result(query, collection):

    get_knowledge = vector_search(query, collection)

    search_result = ""
    for result in get_knowledge:
        search_result += f"text: {result.get('text', 'N/A')}\n"
        # search_result += f"score: {result.get('score', 'N/A')}\n"
    return search_result

In [19]:
# Conduct query with retrival of sources
query = "상품의 정의가 무엇인가요?"
source_information = get_search_result(query, collection)

print(source_information)

text:  . Class Object Diagram
상품팩토리의 상품 정의 순서를 모형화하여 다음과 같이 객체를 생성한다
text:  . 상품조건의 개념
일반적인 금융 상품에는 여러가지 상품의 특성이 기술되어 있다. 아래의 예시를 보면, 가입대상, 가입채널, 가입기간, 납입금액, 이자지급방식, 금리운영방법, 적용이율 등이 상품 특성이다.
상품베이스 컴포넌트에서는 이러한 상품 특성을 상품조건이라고 명명한다. 상품조건은 Product Condition 으로 부르기도 한다. 일반적으로 상품의 특성이 다를수록 상품을 구성하는 상품조건의 차이가 더 많게 된다. 즉, 정기예금의 상품조건 목록과 신용대출의 상품조건 목록은 상이한 상품조건이 다수 존재하고, 정기예금1과 정기예금2의 상품조건을 비교하면 비슷한 상품조건이 더 많다는 것을 알 수 있다.
text:  . 상품의 정의
상품은 "금융기관이 이자 및 수수료 수익 창출을 목적으로 (계약을 기반으로) 고객에게 판매하는 상품 및 서비스”로 정의한다. 이 중, "고객에게 편의 제공을 목적으로 수수료 수익을 얻거나 얻을 것으로 기대하고 판매하는 상품"을 별도로 서비스라고 칭한다.
text:  . 계약의 활동 시점
* 현재 계약한 상품이 계약관계규칙에서 필수적으로 등록되어야 하는 대상 관계가 모두 등록되어 있는지 검증

